This is a tutorial for Python 2.7

# pyphysio library

## 1. Signals and Algorithms

Signal and Algorithm are the two main classes in pyphysio.


### 1.1 Signals in pyphysio

Description of how signals are represented in pyphysio

A signal is an ordered vector of timestamp-value pairs, where the timestamp is the instant at which the measured phenomenon had that value.
In pyphysio a signal is represented by the class **Signal** which extends the numpy.ndarray class.

In this part we will see the different types of signals that can be defined and their properties.

In [ ]:
# import libraries
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# import the Signal classes
from pyphysio import EvenlySignal, UnevenlySignal

#### 1.1.1 EvenlySignal

When the signal is sampled with a fixed sampling frequency it is sufficient to know the timestamp at which the acquisition started and the sampling frequency to reconstruct the timestamp of each sample. This type of signal is represented by the class **EvenlySignal**.

Therefore to create an instance of **EvenlySignal** these are the input attributes needed:
* ``values`` : (unidimensional numpy array) values of the signal;
* ``sampling_freq`` : (float>0) sampling frequency;
* ``start_time`` : (float) temporal reference of the start of the signal. This is optional, if omitted it will set to 0;
* ``signal_nature`` : (string) identifier of the type of the signal. In future releases of pyphysio it will be used to check the appropriateness of the algorithms applied to the signal. Now it is optional and if omitted it will set to ''.

Class functions are provided to facilitate the management and processing of signals:
* `get_...()` type functions can be used to check signal attributes;
* `plot()` will plot the signal using matplotlib;
* `segment_time(t_start, t_stop)` and `segment_idx(idx_start, idx_stop)` can be used to extract a portion of the signal;
* `resample(fout)` can be used to change the sampling frequency.

In [ ]:
# create a signal

## create fake data
np.random.seed(4)
signal_values = np.random.uniform(0, 1, size = 1000)

## set the sampling frequency
fsamp = 100 # Hz

## set the starting time
tstart = 100 # s

## create the Evenly signal
s_fake = EvenlySignal(values = signal_values, sampling_freq = fsamp, signal_nature = 'fake', start_time = tstart)

In [ ]:
## plot
s_fake.plot()

In [ ]:
# chech signal properties
print('Sampling frequency: {}'.format( s_fake.get_sampling_freq() ))
print('Start time:         {}'.format( s_fake.get_start_time() ))
print('End time:           {}'.format( s_fake.get_end_time() ))
print('Duration:           {}'.format( s_fake.get_duration() ))
print('Signal nature:      {}'.format( s_fake.get_signal_nature() ))
print('First ten instants: {}'.format( s_fake.get_times()[0:10] ))

In [ ]:
# check Signal representation
s_fake

In [ ]:
# import data from included examples
from pyphysio import TestData

ecg_data = TestData.ecg()

eda_data = TestData.eda()

In [ ]:
# create two signals
fsamp = 2048
tstart_ecg = 15
tstart_eda = 5
ecg = EvenlySignal(values = ecg_data, sampling_freq = fsamp, signal_nature = 'ecg', start_time = tstart_ecg)
eda = EvenlySignal(values = eda_data, sampling_freq = fsamp, signal_nature = 'eda', start_time = tstart_eda)

In [ ]:
# plot
ax1 = plt.subplot(211)
ecg.plot()
plt.subplot(212, sharex=ax1)
eda.plot()

In [ ]:
# check signal properties
print('ECG')
print('Sampling frequency: {}'.format( ecg.get_sampling_freq() ))
print('Start time:         {}'.format( ecg.get_start_time() ))
print('End time:           {}'.format( ecg.get_end_time() ))
print('Duration:           {}'.format( ecg.get_duration() ))
print('Signal nature:      {}'.format( ecg.get_signal_nature() ))
print('First ten instants: {}'.format( ecg.get_times()[0:10] ))
print('')
print('EDA')
print('Sampling frequency: {}'.format( eda.get_sampling_freq() ))
print('Start time:         {}'.format( eda.get_start_time() ))
print('End time:           {}'.format( eda.get_end_time() ))
print('Duration:           {}'.format( eda.get_duration() ))
print('Signal nature:      {}'.format( eda.get_signal_nature() ))
print('First ten instants: {}'.format( eda.get_times()[0:10] ))

In [ ]:
# resampling
ecg_128 = ecg.resample(fout=128)

ecg.plot()
ecg_128.plot('.-')

#### 1.1.2 UnevenlySignal

Other types of signals, for instance triggers indicating occurrences of heartbeats or events, are series of samples which are not equally temporally spaced. Thus the sampling frequency is not fixed and it is necessary to store the timestamp of each sample. This type of signals is represented by the class **UnevenlySignal**.

Therefore to create an instance of **UnevenlySignal** these are these additional input attributes are needed:
* ``x_values`` : (unidimensional numpy array) information about the temporal position of each sample. Should be of the same size of ``values``;
* ``x_type`` : ('instants' or 'indices') indicate what type of x_values have been used.

Two ways are allowed to define an **UnevenlySignal**:
1. by defining the indexes (`x_type='indices'`): x_values are indices of an array and the instants are automatically computed using the information from the `sampling_frequency` and the `start_time`. 
2. by defining the instants (`x_type='instants'`): x_values are instants and the indices are automatically computed using the information from the `sampling_frequency` and the `start_time`. 

As a general rule, the `start_time` is always associated to the index 0.

An additional class function is provided to transform an **UnevenlySignal** to an **EvenlySignal**:
* `to_evenly()` create an `EvenlySignal` by interpolating the signal with given signal sampling frequency.

In [ ]:
# create a signal

## create fake data
signal_values = np.arange(100)

## create fake indices
idx = np.arange(100)
idx[-1] = 125

## set the sampling frequency
fsamp = 10 # Hz

## set the starting time
tstart = 10 # s

## create an Unevenly signal defining the indices
x_values_idx = idx

s_fake_idx = UnevenlySignal(values = signal_values, sampling_freq = fsamp, signal_nature = 'fake', start_time = tstart,
                       x_values = x_values_idx, x_type = 'indices')

In [ ]:
## create an Unevenly signal defining the indices
x_values_time = idx/fsamp + 10

## set the starting time
tstart = 0

s_fake_time = UnevenlySignal(values = signal_values, sampling_freq = fsamp, signal_nature = 'fake', start_time = tstart,
                       x_values = x_values_time, x_type = 'instants')

In [ ]:
#plot
ax1=plt.subplot(211)
s_fake_idx.plot('.-')
plt.subplot(212, sharex=ax1)
s_fake_time.plot('.-')

In [ ]:
# note that the times are the same but not the starting_time nor the indices:

# check samples instants
print('Instants:')
print(s_fake_idx.get_times())
print(s_fake_time.get_times())

# check samples indices
print('Indices:')
print(s_fake_idx.get_indices())
print(s_fake_time.get_indices())

# check start_time
print('Start time:')
print(s_fake_idx.get_start_time())
print(s_fake_time.get_start_time())

In [ ]:
# chech signal properties
print('Defined by Indices')
print('Sampling frequency: {}'.format( s_fake_idx.get_sampling_freq() ))
print('Start time:         {}'.format( s_fake_idx.get_start_time() ))
print('End time:           {}'.format( s_fake_idx.get_end_time() ))
print('Duration:           {}'.format( s_fake_idx.get_duration() ))
print('Signal nature:      {}'.format( s_fake_idx.get_signal_nature() ))
print('First ten instants: {}'.format( s_fake_idx.get_times()[0:10] ))
print('')
print('Defined by Instants')
print('Sampling frequency: {}'.format( s_fake_time.get_sampling_freq() ))
print('Start time:         {}'.format( s_fake_time.get_start_time() ))
print('End time:           {}'.format( s_fake_time.get_end_time() ))
print('Duration:           {}'.format( s_fake_time.get_duration() ))
print('Signal nature:      {}'.format( s_fake_time.get_signal_nature() ))
print('First ten instants: {}'.format( s_fake_time.get_times()[0:10] ))

In [ ]:
# to_evenly
s_fake_time_evenly = s_fake_time.to_evenly(kind = 'linear')

In [ ]:
s_fake_time_evenly.plot('.-')
s_fake_time.plot('.-')

# check type
print(type(s_fake_time_evenly))
print(type(s_fake_time))

#### 1.1.3 Segmentation of signals

Two general class functions are provided to segment a signal:
1. `segment_time(t_start, t_stop)` is used to extract a portion of the signal between the instants `t_start` and
`t_stop`;
2. `segment_idx(idx_start, idx_stop)` is used to extract a portion of the signal between the indices `idx_start` and `idx_stop`.

The output signal will inherit **`sampling_freq`** and **`signal_nature`** but the **`start_time`** will be set to **`t_start`** or to the instant corresponding to **`idx_start`** accordingly to the method used.

In [ ]:
# segmentation of ES
ecg_segment = ecg.segment_time(45, 54)
eda_segment = eda.segment_time(45, 54)

In [ ]:
# plot
ax1 = plt.subplot(211)
ecg.plot()
ecg_segment.plot('r')

plt.subplot(212, sharex=ax1)
eda.plot()
eda_segment.plot('r')

print(ecg_segment.get_start_time())

In [ ]:
# segmentation of US

s_fake_idx_segment = s_fake_idx.segment_time(10.5, 18)
s_fake_time_segment = s_fake_time.segment_time(10.5, 18)

In [ ]:
# plot
ax1 = plt.subplot(211)
s_fake_idx.plot('.-')
s_fake_idx_segment.plot('.-r')

plt.subplot(212, sharex=ax1)
s_fake_time.plot('.-')
s_fake_time_segment.plot('.-r')

print(s_fake_time_segment.get_start_time())

### 1.2 Algorithms in pyphysio

A signal processing step is a computational function $F$ that operates on input data (a signal) to produce a result. It is characterized by a set of parameters **p** which regulate its behavior.

![algorithm](img/algorithm.png)

*Figure 1: Abstract representation of a processing step.*

1. In pyphysio each processing step is represented by an instance of a class derived from the generic class `Algorithm`.

2. The type of function or algorithm is given by the Class name (e.g. `BeatFromECG` extracts the heartbeats from an ECG signal, `PeakDetection` detects the peaks in the input signal).

3. The parameters of the function/algorithm are the attributes of the created instance.

Therefore, a processing step is defined by creating a new instance of the Class, which is initialized with the given parameters:
```
processing_step = ph.BeatFromECG(parameters)
```
To execute the processing step we need to give as input an instance of the class `Signal`:
```
output = processing_step(input)
```


Algorithms in pyphysio are grouped in four categories:

* Filters : deterministic algorithms that modify the values of the input signal without changing its nature;
* Estimators : algorithms that aim at extracting information from the input signal which is given in output as a signal with a different nature;
* Indicators : algorithms that operate on the signal to provide a scalar value (or metrics)
* Tools : algorithms that can be useful for the signal processing and return as output one or more numpy arrays or scalars.

#### 1.2.1 Filters

Filters return a signal which has the same **`signal_nature`** of the input signal. 

The name *`Filters`* recalls the aim of this algorithms which is in general to increase the Signal/Noise ratio by filtering out the unwanted components in a signal (e.g high frequency noise).

In [ ]:
# create a Filter
import pyphysio.filters.Filters as flt

lowpass_50 = flt.IIRFilter(fp=50, fs=75, ftype='ellip')

In [ ]:
# help inline
#?flt.IIRFilter

In [ ]:
# check parameters
print(lowpass_50)
# OR
print(lowpass_50.get())

In [ ]:
# apply a Filter
ecg_filtered = lowpass_50(ecg)

In [ ]:
#plot
ecg.plot()
ecg_filtered.plot()

In [ ]:
# check output type
ecg.get_signal_nature()

#### 1.2.2 Estimators
Estimators are algorithms which aim at extracting the information of interest from the input signal, thus returning a new signal which has a different **`signal_nature`**. 

The name *`Estimators`* recalls the fact that the information extraction depends on the value of the algorithm parameters which might not be known *a-priori*. Thus the result should be considered as an estimate of the real content of information of the input signal.

In [ ]:
# create an Estimator
import pyphysio.estimators.Estimators as est

ibi_ecg = est.BeatFromECG()

In [ ]:
# check parameters
ibi_ecg

In [ ]:
# apply an Estimator
ibi = ibi_ecg(ecg_filtered)

In [ ]:
# plot
ax1 = plt.subplot(211)
ecg.plot()

plt.subplot(212, sharex=ax1)
ibi.plot()

In [ ]:
# check output type
ibi.get_signal_nature()

#### 1.2.3 Indicators

Indicators are algorithm which extract a metrics (scalar value) from the input signal, for instance a statistic (average).

Three types of indicators are provided in **`pyphysio`**:
* Time domain indicators: comprising simple statistical indicators and other metrics that can be computed on the signal values;
* Frequency domain indicators: metrics that are computed on the Power Spectrum Density (PSD) of the signal;
* Non-linear indicators: complex indicators that are computed on the signal values (e.g. Entropy).

In [ ]:
# create an Indicator
import pyphysio.indicators.TimeDomain as td_ind
import pyphysio.indicators.FrequencyDomain as fd_ind

In [ ]:
rmssd = td_ind.RMSSD()
HF = fd_ind.PowerInBand(interp_freq=4, freq_max=0.4, freq_min=0.15, method = 'ar')

In [ ]:
# check parameters
print(rmssd)
print(HF)

In [ ]:
# apply an Indicator
rmssd_ = rmssd(ibi)
HF_ = HF(ibi)

print(rmssd_)
print(HF_)

In [ ]:
# check output type
print(type(rmssd_))
print(type(HF_))

#### 1.2.4 Tools

This is a collection of useful algorithms that can be used for signal processing. 

These algorithms might return scalar values or numpy arrays.

In [ ]:
# create a Tool
import pyphysio.tools.Tools as tll

compute_psd = tll.PSD(method='ar', interp_freq = 4)

In [ ]:
# check parameters
compute_psd

In [ ]:
# apply a Tool
frequencies, power = compute_psd(ibi)

plt.plot(frequencies, power)
plt.show()